# 1. Установка зависимостей

In [ ]:
# Установка зависимостей и Chrome
!pip install selenium
!apt-get update
!apt-get install -y wget curl unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt --fix-broken install -y
!pip install chromedriver-autoinstaller

# 2. Загрузка ключевых слов

In [ ]:
!pip install gdown chardet

import gdown
import chardet

# Скачивание файла с ключевыми словами с Google Диска
file_url = 'https://drive.google.com/uc?id=***'  # Укажите ID файла из ссылки
# (Мой диск: напротив keywords.txt 3 точки - Открыть доступ - Все, у кого есть ссылка - Копировать ссылку
# - Вставить в код все, что находится в ссылке вместо *** https://drive.google.com/file/d/***/view?usp=sharing)
file_name = 'keywords.txt'

def download_keywords_file(url, output):
    print("Загружаем файл ключевых слов...")
    gdown.download(url, output, quiet=False)
    print(f"Файл сохранён как {output}")

def load_keywords(file_name):
    print("Читаем ключевые слова из файла...")
    with open(file_name, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        print(f"Определённая кодировка файла: {encoding}")

    try:
        content = raw_data.decode(encoding)
        # Разделение по строкам
        keywords = [line.strip() for line in content.splitlines() if line.strip()]
        return keywords
    except Exception as e:
        print(f"Ошибка при чтении файла: {e}")
        return []

    # Загрузка ключевых слов
download_keywords_file(file_url, file_name)
keywords = load_keywords(file_name)

Загружаем файл ключевых слов...


Downloading...
From: https://drive.google.com/uc?id=1bsFU4q_jPQ-J0V1M4zB1OJZona2-ppRE
To: /content/keywords.txt
100%|██████████| 19.0k/19.0k [00:00<00:00, 28.1MB/s]

Файл сохранён как keywords.txt
Читаем ключевые слова из файла...
Определённая кодировка файла: utf-8


# 3. Сбор данных с Google Новостей

In [ ]:
import asyncio
import aiohttp
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import re
import os

# Монтируем Google Drive
from google.colab import drive as gc_drive
gc_drive.mount('/content/drive')

# Проверка и создание папки для сохранения данных на Google Диске
FOLDER_NAME = "TextScope"
BASE_DRIVE_PATH = "/content/drive/MyDrive"
folder_path = os.path.join(BASE_DRIVE_PATH, FOLDER_NAME)

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Полный путь к файлу CSV
DRIVE_CSV_PATH = os.path.join(folder_path, "gnews_data.csv")

# 4. Парсинг статьи и извлечение данных

In [ ]:
import csv

# Функция для извлечения даты из текста
def extract_date_with_regex(text):
    iso = re.search(r"\b(\d{4}-\d{2}-\d{2})\b", text)
    if iso:
        return iso.group(1)
    rus_pattern = r"\b(\d{1,2})\s+(января|февраля|марта|апреля|мая|июня|июля|августа|сентября|октября|ноября|декабря)\s+(\d{4})\b"
    m = re.search(rus_pattern, text, re.IGNORECASE)
    if m:
        day, month, year = m.groups()
        months = {"января":"01","февраля":"02","марта":"03","апреля":"04","мая":"05","июня":"06","июля":"07","августа":"08","сентября":"09","октября":"10","ноября":"11","декабря":"12"}
        return f"{year}-{months[month.lower()]}-{int(day):02d}"
    return None

# Функция сохранения данных в CSV
def save_to_drive_colab_append(new_data: list, path=DRIVE_CSV_PATH):
    df_new = pd.DataFrame(new_data)
    if 'keyword' in df_new.columns:
        cols = ['keyword'] + [c for c in df_new.columns if c != 'keyword']
        df_new = df_new[cols]
    file_exists = os.path.isfile(path)
    try:
        df_new.to_csv(path, mode='a', header=not file_exists, index=False, encoding='utf-8-sig', quoting=csv.QUOTE_ALL, quotechar='"')
        print(f"{len(df_new)} строк(а) дописано в {path}.")
    except UnicodeEncodeError as e:
        print(f"Ошибка кодирования при сохранении CSV: {e}")

# 5. Получение реального URL после редиректа с Selenium

In [ ]:
# Функция для получения реального URL с помощью Selenium
async def resolve_redirect_selenium_async(url, delay=1):
    def resolve_with_selenium():
        try:
            options = Options()
            options.add_argument("--headless")
            options.add_argument("--disable-gpu")
            options.add_argument("--no-sandbox")
            service = Service()
            driver = webdriver.Chrome(service=service, options=options)
            driver.get(url)
            time.sleep(delay)
            real_url = driver.current_url
            driver.quit()
            return real_url
        except Exception as e:
            print(f"Ошибка Selenium для {url}: {e}")
            return None
    return await asyncio.to_thread(resolve_with_selenium)

# 6. Основной процесс обработки новостей

In [ ]:
# Основной процесс получения новостей по ключевому слову
async def process_news_async(keyword: str):
    news = await get_google_news_rss_async(keyword)
    async with aiohttp.ClientSession() as session:
        for idx, item in enumerate(news, 1):
            print(f"[{idx}/{len(news)}] {item['title']}")
            real_url = await resolve_redirect_selenium_async(item["link"])
            article = await parse_article_async(real_url, session, rss_title=item['title'], rss_pubDate=item['pubDate'])
            if article:
                record = {"keyword": keyword, **article}
                save_to_drive_colab_append([record])
            else:
                print(f"Пропуск: не удалось получить данные для {item['link']}")

# Запуск основного процесса
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()

    keywords_list = keywords  # Ваш список ключевых слов

    loop = asyncio.get_event_loop()
    for kw in keywords_list:
        print(f"\n=== Поиск: {kw} ===")
        loop.run_until_complete(process_news_async(kw))